<a href="https://colab.research.google.com/github/wantwotwee/AI-Agent/blob/main/AI_Agent_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!git config --global user.email "wa0001at@gmail.com"
!git config --global user.name "wantwotwee"

In [7]:
!git clone https://github.com/wantwotwee/AI-Agent.git
%cd AI-Agent

Cloning into 'AI-Agent'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (3/3), done.
/content/AI-Agent


In [12]:
!ls /content/

AI-Agent  sample_data


**Welcome to Langchain Tutorial created by Wan. Langchain allows you to create RAG AI Agents. It helps you to link different AI tools to do specific tasks. My ultimate goal with this tutorial is to create an AI Agent to automatically create media content in social media format. I will be exploring and building on the end product with whatever I learnt on this journey.**

**Step 1: Dependencies to Download**

In [ ]:
!pip install langchain
!pip install langchain-community
!pip install openai

**Step 2: Get an API Key from OpenAI or DeepSeek. I got the DeepSeek API because it's 10 times cheaper for now. Now, test if it works. If there's no error, it works!**

In [ ]:
from openai import OpenAI

client = OpenAI(api_key="sk-c05ffe4ebb8d48a789b03d65f3f1f17e", base_url="https://api.deepseek.com")

response = client.chat.completions.create(
    model="deepseek-chat",
    messages=[
        {"role": "system", "content": "You are a helpful assistant"},
        {"role": "user", "content": "Hello"},
    ],
    stream=False
)

print(response.choices[0].message.content)

Hello! How can I assist you today? 😊


**Step 3: Install additional dependencies that's required by the AI model.**

In [ ]:
pip install langchain-deepseek

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 32.2 MB/s eta 0:00:00


**Step 4: Input your secret API Key here as the environment**

In [ ]:
%env DEEPSEEK_API_KEY=sk-c05ffe4ebb8d48a789b03d65f3f1f17e

env: DEEPSEEK_API_KEY=sk-c05ffe4ebb8d48a789b03d65f3f1f17e


**Step 5: Choose the Large Language Model to use. This is the "brain". DeepSeek has "deepseek-reasoner" and "deepseek-chat".**

In [ ]:
from langchain_deepseek import ChatDeepSeek
llm = ChatDeepSeek(model="deepseek-chat")

**Step 6: Test if the LLM works by giving any chat prompt.**

In [ ]:
response = llm.predict("Give me a nickname for Singapore")
print(response)

Here are a few creative nicknames for Singapore, inspired by its unique characteristics:  

1. **"The Lion City"** – From its Malay name *Singapura* ("Lion City").  
2. **"The Garden City"** – For its lush greenery and urban parks.  
3. **"The Little Red Dot"** – A playful (sometimes self-deprecating) nod to its small size on world maps.  
4. **"Asia’s Fine City"** – A pun on its strict laws (fines for littering, etc.) and polished reputation.  
5. **"The Melting Pot"** – Reflecting its multicultural society.  
6. **"The Smart Nation"** – Highlighting its tech-forward, efficient infrastructure.  

Which one resonates most with you? Or would you like something more personalized? 😊


**Step 7: Create a PROMPT template.**

In [ ]:
from langchain.prompts import PromptTemplate

prompt_template_name = PromptTemplate(
    input_variables = ['country'],
    template = "Give me a nickname for {country}. Only one nickname please."
)

prompt_template_name.format(country="Russia")

'Give me a nickname for Russia. Only one nickname please.'

In [ ]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=prompt_template_name)
chain.run("Russia")

'"Mother Russia"'

**Step 8: LangChain allows you to do Simple Sequential Chains. The output from one prompt can be used for another prompt.**

In [ ]:
from langchain.prompts import PromptTemplate

prompt_template_name = PromptTemplate(
    input_variables = ['country'],
    template = "Give me a nickname for {country}. Only one nickname please."
)

prompt_template_history = PromptTemplate(
    input_variables = ['nickname'],
    template = "Give me some history behind {nickname}. Return it as one short paragraph."
)

country_chain = LLMChain(llm=llm, prompt=prompt_template_name)
history_chain = LLMChain(llm=llm, prompt=prompt_template_history)

In [ ]:
from langchain.chains import SimpleSequentialChain

chain = SimpleSequentialChain(chains=[country_chain, history_chain])
response = chain.run("Turkey")
print(response)

"Land of the Crescent Moon" 🌙 is a poetic nickname often linked to regions where the crescent moon holds cultural or historical significance, such as the Ottoman Empire, which adopted it as a symbol in the 14th century. The crescent moon became a prominent emblem on flags, coins, and architecture, representing power and faith, particularly in Islamic civilizations. It also appears in ancient Mesopotamian and Turkic traditions, symbolizing deities, renewal, or conquest. Today, the crescent remains iconic in flags like Turkey’s and Pakistan’s, tying modern identities to a rich, celestial heritage.


**Step 9: LangChain also allows you to do Sequential Chains. This is the same as the previous step, but the final output contains all the inputs & inputs entered into the model**

In [ ]:
from langchain.prompts import PromptTemplate

prompt_template_name = PromptTemplate(
    input_variables = ['country'],
    template = "Give me a nickname for {country}. Only one nickname please."
)

country_chain = LLMChain(llm=llm, prompt=prompt_template_name, output_key="nickname")


prompt_template_history = PromptTemplate(
    input_variables = ['nickname'],
    template = "Give me some history behind {nickname}. Return it as one short paragraph."
)

history_chain = LLMChain(llm=llm, prompt=prompt_template_history, output_key="history")

In [ ]:
from langchain.chains import SequentialChain

chain = SequentialChain(
    chains = [country_chain, history_chain],
    input_variables = ['country'],
    output_variables=['nickname', 'history'])

chain({'country': "Turkey"})

<ipython-input-47-e5ffe7853cbb>:8: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain({'country': "Turkey"})


{'country': 'Turkey',
 'nickname': '"Anatolia" – a nod to Turkey\'s historical and geographical roots in the region.',
 'history': 'Anatolia, known historically as Asia Minor, is the vast peninsula that forms the heartland of modern-day Turkey, bridging Europe and Asia. This region has been a cradle of civilizations for millennia, from the ancient Hittites and Phrygians to the Greeks, Romans, and Byzantines, before becoming the core of the Ottoman Empire. Its strategic location made it a crossroads of trade, culture, and conflict, shaping its rich heritage. The name "Anatolia" itself derives from the Greek *Anatolē*, meaning "east" or "sunrise," reflecting its position as the eastern frontier of the classical world. Today, it remains a symbol of Turkey’s deep historical and geographical roots, blending diverse influences into a unique cultural identity.'}

**Step 10:

In [ ]:
from langchain_community.tools import BraveSearch
api_key = "API KEY"

ValidationError: 1 validation error for AskNewsAPIWrapper
  Value error, Did not find asknews_client_id, please add an environment variable `ASKNEWS_CLIENT_ID` which contains it, or pass `asknews_client_id` as a named parameter. [type=value_error, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/value_error